# Gestor de Dataset y Etiquetado para YOLO Segmentación

Este cuaderno permite agregar nuevas imágenes al dataset, generar etiquetas de segmentación y preparar el modelo para el re-entrenamiento.

## 1. Configuración de Rutas y Dependencias


In [ ]:
import os
import shutil
import yaml
import cv2
from ultralytics import YOLO

# Rutas principales
BASE_DIR = '/home/adrian/Desktop/Aprendizaje/practica_final'
DATASET_PATH = os.path.join(BASE_DIR, 'dataset')
YAML_PATH = os.path.join(DATASET_PATH, 'data.yaml')

# Asegurar que el archivo data.yaml existe
if not os.path.exists(YAML_PATH):
    print("Error: Ejecuta primero el notebook 'test.ipynb' para inicializar el data.yaml")


## 2. Agregar Nuevo Producto (SKU)

Utiliza esta función si necesitas registrar un SKU que no existe en el sistema.


In [ ]:
def register_new_sku(sku_name):
    with open(YAML_PATH, 'r') as f:
        data = yaml.safe_load(f)
    
    # Obtener el siguiente ID disponible
    ids = list(data['names'].keys())
    next_id = max(ids) + 1 if ids else 0
    
    # Agregar si no existe
    if sku_name not in data['names'].values():
        data['names'][next_id] = sku_name
        with open(YAML_PATH, 'w') as f:
            yaml.dump(data, f)
        print(f"Registrado {sku_name} con ID: {next_id}")
        return next_id
    else:
        for k, v in data['names'].items():
            if v == sku_name:
                print(f"El SKU {sku_name} ya existe con ID: {k}")
                return k

# Ejemplo de uso:
# new_id = register_new_sku('NUEVO-LAPTOP-X')


## 3. Agregar Imagen y Etiqueta de Segmentación

Para YOLO segmentación, la etiqueta es un archivo `.txt` con el formato:
`<class_id> <x1> <y1> <x2> <y2> ... <xn> <yn>`
Donde las coordenadas están normalizadas (divididas por el ancho/alto de la imagen).


In [ ]:
def add_labeled_image(image_path, sku_id, polygon_coords, split='train'):
    """
    image_path: Ruta a la imagen original
    sku_id: El ID numerico del SKU
    polygon_coords: Lista de tuplas/listas [(x1,y1), (x2,y2)...] en coordenadas pixel
    """
    img = cv2.imread(image_path)
    h, w, _ = img.shape
    
    # Nombre base del archivo
    filename = os.path.basename(image_path)
    basename = os.path.splitext(filename)[0]
    
    # Copiar imagen al dataset
    target_img_path = os.path.join(DATASET_PATH, f'images/{split}', filename)
    shutil.copy(image_path, target_img_path)
    
    # Generar etiqueta YOLO segmentación (normalizada)
    normalized_coords = []
    for x, y in polygon_coords:
        normalized_coords.append(f"{x/w:.6f} {y/h:.6f}")
    
    label_content = f"{sku_id} {' '.join(normalized_coords)}\n"
    
    label_file_path = os.path.join(DATASET_PATH, f'labels/{split}', f"{basename}.txt")
    with open(label_file_path, 'w') as f:
        f.write(label_content)
    
    print(f"Imagen y etiqueta guardadas para: {filename}")

# Ejemplo:
# coords = [(100, 150), (400, 150), (400, 500), (100, 500)] # Rectángulo de ejemplo
# add_labeled_image('mi_laptop.jpg', 0, coords)


## 4. Iniciar Entrenamiento

Una vez que tengas suficientes imágenes (se recomienda al menos 20-50 por SKU), ejecuta esta celda.


In [ ]:
# Cargar el modelo base para transfer learning
model = YOLO('yolo11n-seg.pt')

# Entrenar
results = model.train(
    data=YAML_PATH,
    epochs=100,
    imgsz=640,
    batch=16,
    name='sku_segmentation_model'
)

print("Entrenamiento completado. Los resultados están en 'runs/segment/sku_segmentation_model'")
